In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("Example1")
    .getOrCreate()
)

sc = spark.sparkContext


In [15]:
# 1. lectura de datos
rdd = sc.textFile('./data/casoDePrueba1.txt')

# Convirtiendo datos en tupla
rdd = (rdd
       .map(lambda line: line.split(";"))
       .map(lambda x: (x[0], x[1], float(x[2])))
)

In [20]:
# Seleccionamos persona y gasto en caso este sea con TDC en otro caso se coloca 0
rdd_mapped = rdd.map(lambda x: (x[0], x[2] if x[1] == 'Tarjeta de crédito' else 0))

# Sumarizamos los gastos por persona
rdd_reduced = rdd_mapped.reduceByKey(lambda x, y: x + y)

In [22]:
# Generamos nuevo rdd con separador
rdd_save = rdd_reduced.map(lambda x: f"{x[0]};{x[1]}")

# almacenamos data en path
out_path = './out/personaGastosConTarjetaCredito.txt'
rdd_save.saveAsTextFile(out_path)
